In [1]:
import os
import time
import re
import random
import concurrent.futures
import warnings
import datetime
import pandas as pd
import traceback
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent
from selenium.webdriver.chrome.service import Service

from tqdm.notebook import tqdm

from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import Future
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
#Driver
chrome_options = Options()
try :
    path = r"C:\Cours\Python\chromedriver_win32\chromedriver.exe"
    #chrome_options.add_argument("--headless")
    chrome = webdriver.Chrome(executable_path= path ,options=chrome_options)
    chrome.maximize_window()
except:
    path = r"C:\Users\w.grasina\OneDrive - Adelphon\Bureau\TRAVAUX\Crawls\chromedriver.exe"
    #chrome_options.add_argument("--headless")
    chrome = webdriver.Chrome(executable_path= path ,options=chrome_options)
    chrome.maximize_window()
    

In [3]:
chrome.get("https://www.gites-de-france.com/fr/search?destination=Corse&regions=18&seed=79f57e24")

In [4]:
#Regions
lecture_regions = chrome.find_element(By.XPATH,'//*[@id="main"]/article/section[1]/div/div/section[2]')
lecture_regions = lecture_regions.find_element(By.TAG_NAME,"svg")
lecture_regions = lecture_regions.find_elements(By.TAG_NAME,"a")
regions = [region.get_attribute("href")
          for region in lecture_regions]
regions = ["https://www.gites-de-france.com"+region["animVal"] for region in regions]
regions

['https://www.gites-de-france.com/fr/search?destination=Corse&regions=18',
 'https://www.gites-de-france.com/fr/search?destination=Alsace&regions=72212',
 'https://www.gites-de-france.com/fr/search?destination=Provence-Alpes-C%C3%B4te%20d%27Azur&regions=17',
 'https://www.gites-de-france.com/fr/search?destination=Lorraine&regions=72224',
 'https://www.gites-de-france.com/fr/search?destination=Rh%C3%B4ne-Alpes&regions=72231',
 'https://www.gites-de-france.com/fr/search?destination=Franche-Comt%C3%A9&regions=72221',
 'https://www.gites-de-france.com/fr/search?destination=Champagne-Ardenne&regions=72219',
 'https://www.gites-de-france.com/fr/search?destination=Bourgogne&regions=72216',
 'https://www.gites-de-france.com/fr/search?destination=Provence-Alpes-C%C3%B4te%20d%27Azur&regions=17',
 'https://www.gites-de-france.com/fr/search?destination=Languedoc%20Roussillon&regions=72235',
 'https://www.gites-de-france.com/fr/search?destination=Auvergne&regions=72214',
 'https://www.gites-de-fran

In [5]:
regions.append("https://www.gites-de-france.com/fr/search?destination=La%20R%C3%A9union&regions=4")
regions.append("https://www.gites-de-france.com/fr/search?destination=Guadeloupe&regions=1")
regions.append("https://www.gites-de-france.com/fr/search?destination=Martinique&regions=2")

In [6]:
regions = list(set(regions))

In [7]:
with open("Regions.txt",'a',encoding="utf-8") as file:
    for region in regions:
        print(region,file=file)

In [ ]:
#with open("urls.txt","w",encoding ="utf-8") as file:
    #print("Region\tUrl",file=file)

In [13]:
for region in tqdm(regions[17:]) :
    chrome.get(region)
    time.sleep(1)
    #Dernière page
    last_page = chrome.find_element(By.CSS_SELECTOR,"li[class='pager__item pager__item--last']")
    num_last_page = int(last_page.find_element(By.TAG_NAME,"a").get_attribute("href").split("page=")[1])
    
    #1ere page
    urls = chrome.find_elements(By.CSS_SELECTOR,"a[class='g2f-accommodationTile-link']")
    urls = [url.get_attribute("href") for url in urls]
    for url in urls :
        with open("urls.txt","a",encoding ="utf-8") as file:
            print(region+"\t"+url,file=file)
    #Autres pages
    for page in range(1,num_last_page+1) :
        chrome.get(region+"&page={}".format(page))
        time.sleep(1)
        urls = chrome.find_elements(By.CSS_SELECTOR,"a[class='g2f-accommodationTile-link']")
        urls = [url.get_attribute("href") for url in urls]
        for url in urls :
            with open("urls.txt","a",encoding ="utf-8") as file:
                print(region+"\t"+url,file = file)

  0%|          | 0/11 [00:00<?, ?it/s]

In [8]:
chrome.quit()